# Day 8
## Problem 1
Count how many times do digits 1, 4, 7, 8 appear.
- input has many sets each of which consists of; 
    - 1) 10 unique signal patterns, 
    - 2) a | delimiter, and 
    - 3) the 4 digit output value.
- Each set is coded differently.

### Approach to take
1. In this probelm, it is OK to see only the part after |.
    -> use numpy.loadtxt, delimiter |, use converters and apply aplit by a space to each colon to make 2 separate list in size 10 and 4.
2. prepare a variable to count up
3. count an element whose length is either 2, 3, 4, 7
4. repeat 3 for the number of input sets.

### Let's try

In [28]:
"""
the following is my record of attempt to use numpy but did not work.
# import numpy as np
# test = np.loadtxt('example.txt', delimiter='|', converters = {0: lamda s: s.split(' '), 1: lamda s: s.split(' ')}, dtype='str', encoding='utf8')
# print(test)
# it did not work, probably because one set is devided by | into 2 lines?
# also, even if it is in 1 line, if the column 0 and 1 are in different shape, it seems not working.
"""

def count_four_digits(input):
    with open(input, 'r') as f:
        test = f.read()
    test_lines = test.split('\n')
    test_output = test_lines[1::2] # from the 2nd line, every 2nd lines
    test_output_digit = [i.split(' ') for i in test_output]

    # print(test_output, "\n", test_output_digit)

    # it seems `in` is useful to check against multiple values
    uniques=[2,3,4,7]
    count = 0

    for i in test_output_digit:
        for j in i:
            if len(j) in uniques:
                count += 1

    return(count)



In [23]:
count_four_digits('input.txt')

578

Oh no, I missed the part "(The entry is wrapped here to two lines so it fits; in your notes, it will all be on a single line.)"...

In [47]:
def count_four_digits_oneline(input):
    with open(input, 'r') as f:
        test = f.read()
    test_lines = test.split('\n')
    # print(test_lines)
    # print(test_lines[0].split(' ')[-4:]) # this seems working to show the last 4 outputs of the 1st set.
    test_output_digit = [i.split(' ')[-4:] for i in test_lines]
    # print(test_output_digit)

    uniques=[2,3,4,7]
    count = 0

    for i in test_output_digit:
        for j in i:
            if len(j) in uniques:
                count += 1

    return(count)  

In [48]:
count_four_digits_oneline('example_one_line.txt')

26

OK, now it seems working. Let's try again!

In [49]:
count_four_digits_oneline('input.txt')

375

## Problem 2

Find out signal patterns for each set, determine the number for 4 digits, and add up all.

### Approach

From 0 to 9, each position appears for the following times.

a: 8
b: 6 (unique)
c: 8
d: 7
e: 4 (unique)
f: 9 (unique)
g: 7

This means, by counting how many times each character in the left side of | appears, position b, e, and f are known.

As seen in Problem 1, the following digit uses unique number of characters. 
1: 2 positions (c, f)
4: 4 positions (b, c, d, f)
7: 3 positions (a, c, f)
8: all positions

So let's try in the following way:

1. Divide one line (one input set) into 10 elements on the left side (`left`) and 4 elements on the right side (`right`) of the pipe '|'.
2. Prepare the followings: 
    - a list of 7 zeros to store the count of each character is used on the left side. `count_char[]`
    - a list of 4 FALSE elements to record if there are 2,3,4 or 7 on the right side. `four_digits_fix[]`
    - a list of 4 zeros to store digits expressed on the right side `four_digits[]`
    - a list of [a-g] `a-g`
    - empty lists `group_five`, `group_six`, `output`
3. Measure length of the 4 elements on the right side:
    - if it is either 2,3,4 or 7, 
        - assign corresponding digit in `four_digits`
        - assign TRUE in `four_digits_fix`, 
            - if the length is 2,
                - assign the element to `actual_c_f`. 
        - if all the elements are assigned digit (== all elements in `four_digits_fix` are TRUE),
            - go to step 8.
    - else (if any element is none of 2,3,4 or 7),
        - amend the element (`right[]`) to `group_five` or `group_six` for each length.
4. Count how many times each character appears in `left` and store counts in `count_char`. 
5. Find `i` where `len(count_char[i]) == 4, 6, and 9` and for each, assign each `i` *in alphabet* to the following: 
    - `actual_e` (len() == 4)
    - `actual_b` (len() == 6)
    - `actual_f` (len() == 9)
6. If `len(group_five)` > 0
    - For each element, find 
        - if it contains `actual_b` -> assign 5 in `four_digit`, true in `four_digit_fix`
        - elif it contains `actual_e` -> assign 2 in `four_digit`, true in `four_digit_fix`
        - else assign 3 in `four_digit`, true in `four_digit_fix`
    - If 
7. If `len(group_six)` > 0
        if `actual_c_f` is not assigned any value (== 0)
            - measure length of elements of `left` until it finds the one with the length of 2 and assign it to `actual_c_f`
    - For each element, find 
        - if it does not contain `actual_e` -> assign 9 in `four_digit`, true in `four_digit_fix`
        - elif it contains `actual_c_f` -> assign 0 in `four_digit`, true in `four_digit_fix` 
        - else -> assign 6 in `four_digit`, true in `four_digit_fix`    
8. calculate the value that should have been shown by the 4 elements and amend to `output`
9. repeat 1-8 for all the lines
10. return sum(output) 
 
Not sure if this works, but let's try..

In [93]:
a_g = 'abcdefg'
output_values = []

with open('example_one_line.txt', 'r') as f:
    test = f.read()
test_lines = test.split('\n')
# print("how many input sets: ",len(test_lines))
right = [i.split(' ')[-4:] for i in test_lines]
left = [i.split(' ')[:10] for i in test_lines]
# print(right[0])
# print(left[0])


count_char = [0] * 7
four_digits_fix = [bool(False)] * 4
four_digits = [0] * 4
group_five = []
group_six = []

digit_dict = {2:1,3:7,4:4,7:8}
# print(digit_dict[7])


for i in range(0,len(test_lines)):
    for j in range(0,4):
        if len(right[i][j]) in [2,3,4,7]:
            four_digits[j] = digit_dict[len(right[i][j])]
            four_digits_fix[j] = True
            if len(right[i][j]) == 2:
                actual_c_f = right[i][j]
        elif len(right[i][j]) == 5:
            group_five.append(j)
        else: 
            group_six.append(j)
        if sum(four_digits_fix) == 4:
            print("step 8")
    # print(four_digits)
    # print(four_digits_fix)
    # print("group_five: ", group_five)
    # print("group_six: ", group_five)

    for k in range(0,10):
        for char in range(0,7):
            if a_g[char] in left[i][k]: 
                count_char[char] += 1
        if len(left[i][k]) == 2:
            actual_c_f = left[i][k]
    # print("count_char: ", count_char)

    three_char = 0
    for char in range(0,7):
        if count_char[char] == 4:
            actual_e = a_g[char]
            three_char += 1
        elif count_char[char] == 6:
            actual_b = a_g[char]
            three_char += 1
        elif count_char[char] == 9:
            actual_f = a_g[char]
            three_char += 1
        if three_char == 3:
            break
    # print("actual_e: ", actual_e)
    # print("actual_b: ", actual_b)
    # print("actual_f: ", actual_f)

    if len(group_five) > 0:
        for elem in group_five:
            if actual_b in right[i][elem]:
                four_digits[elem] = 5
                four_digits_fix[elem] = True
            elif actual_e in right[i][elem]:
                four_digits[elem] = 2
                four_digits_fix[elem] = True
            else:
                four_digits[elem] = 3
                four_digits_fix[elem] = True
    if len(group_six) > 0:
        for elem in group_six:
            if actual_e not in right[i][elem]:
                four_digits[elem] = 9
                four_digits_fix[elem] = True
            elif actual_c_f in right[i][elem]:
                four_digits[elem] = 0
                four_digits_fix[elem] = True
            else:
                four_digits[elem] = 6
                four_digits_fix[elem] = True   

    # print(four_digits)
    # print(four_digits_fix)

    output = four_digits[0]*1000+four_digits[1]*100+four_digits[2]*10+four_digits[3]
    # print(output)

    output_values.append(output)

    count_char = [0] * 7
    four_digits_fix = [bool(False)] * 4
    four_digits = [0] * 4
    group_five = []
    group_six = []

print(output_values)
print("in total: ", sum(output_values))


step 8
step 8
[8394, 9781, 1197, 9361, 4873, 8418, 4548, 1625, 8717, 4315]
in total:  61229


In [96]:
a_g = 'abcdefg'
output_values = []

with open('input.txt', 'r') as f:
    test = f.read()
test_lines = test.split('\n')
right = [i.split(' ')[-4:] for i in test_lines]
left = [i.split(' ')[:10] for i in test_lines]

count_char = [0] * 7
four_digits_fix = [bool(False)] * 4
four_digits = [0] * 4
group_five = []
group_six = []

digit_dict = {2:1,3:7,4:4,7:8}


for i in range(0,len(test_lines)):
    for j in range(0,4):
        if len(right[i][j]) in [2,3,4,7]:
            four_digits[j] = digit_dict[len(right[i][j])]
            four_digits_fix[j] = True
            if len(right[i][j]) == 2:
                actual_c_f = right[i][j]
        elif len(right[i][j]) == 5:
            group_five.append(j)
        else: 
            group_six.append(j)
        if sum(four_digits_fix) == 4:
            print("step 8")

    for k in range(0,10):
        for char in range(0,7):
            if a_g[char] in left[i][k]: 
                count_char[char] += 1
        if len(left[i][k]) == 2:
            actual_c_f = left[i][k]

    three_char = 0
    for char in range(0,7):
        if count_char[char] == 4:
            actual_e = a_g[char]
            three_char += 1
        elif count_char[char] == 6:
            actual_b = a_g[char]
            three_char += 1
        elif count_char[char] == 9:
            actual_f = a_g[char]
            three_char += 1
        if three_char == 3:
            break

    if len(group_five) > 0:
        for elem in group_five:
            if actual_b in right[i][elem]:
                four_digits[elem] = 5
                four_digits_fix[elem] = True
            elif actual_e in right[i][elem]:
                four_digits[elem] = 2
                four_digits_fix[elem] = True
            else:
                four_digits[elem] = 3
                four_digits_fix[elem] = True
    if len(group_six) > 0:
        for elem in group_six:
            if actual_e not in right[i][elem]:
                four_digits[elem] = 9
                four_digits_fix[elem] = True
            elif actual_c_f[0] in right[i][elem] and actual_c_f[1] in right[i][elem]: # the order of alphabet can be switched!!
                four_digits[elem] = 0
                four_digits_fix[elem] = True
            else:
                four_digits[elem] = 6
                four_digits_fix[elem] = True   

    output = four_digits[0]*1000+four_digits[1]*100+four_digits[2]*10+four_digits[3] 
    output_values.append(output)

    count_char = [0] * 7
    four_digits_fix = [bool(False)] * 4
    four_digits = [0] * 4
    group_five = []
    group_six = []

print(output_values)
print("in total: ", sum(output_values))

how many input sets:  200
step 8
step 8
step 8
step 8
step 8
step 8
step 8
[6946, 7058, 1995, 7322, 1469, 6339, 1598, 8036, 9139, 8334, 3075, 9747, 7788, 4440, 4804, 1540, 163, 7693, 2146, 2985, 7374, 1000, 8181, 1839, 9574, 6036, 122, 4650, 9508, 8664, 4338, 9953, 7034, 1838, 8301, 6337, 9190, 6446, 1012, 2741, 9875, 1344, 2478, 9788, 4762, 8841, 3703, 9676, 1000, 2974, 9873, 6047, 2748, 8231, 1544, 8117, 9448, 8617, 4435, 3653, 6778, 4420, 3257, 5500, 8699, 2503, 5619, 3048, 1413, 431, 4400, 1106, 5459, 2879, 4183, 7054, 606, 5770, 8374, 1914, 7480, 2986, 2009, 4138, 7731, 8547, 1078, 5441, 1402, 3772, 5635, 7904, 127, 7255, 7788, 9779, 1498, 115, 8920, 8326, 381, 154, 9037, 9541, 7788, 2781, 9877, 4935, 4880, 6329, 5853, 4918, 8571, 3105, 8762, 6018, 5117, 4345, 7140, 8856, 8497, 1294, 7101, 6381, 6268, 4656, 6404, 6089, 5718, 4826, 6420, 3501, 8802, 4227, 9246, 441, 2008, 479, 3919, 5765, 5745, 7563, 318, 6517, 1715, 6511, 8023, 3871, 2824, 1771, 1048, 5323, 5783, 537, 1186, 7584, 

Yes, finally it worked!!!!
Below is defined function. 

I should be able to modularize smaller parts into separate functions...

In [98]:
def sum_actual_output_values(input):
    a_g = 'abcdefg'
    output_values = []

    with open(input, 'r') as f:
        test = f.read()
    test_lines = test.split('\n')
    right = [i.split(' ')[-4:] for i in test_lines]
    left = [i.split(' ')[:10] for i in test_lines]

    count_char = [0] * 7
    four_digits_fix = [bool(False)] * 4
    four_digits = [0] * 4
    group_five = []
    group_six = []

    digit_dict = {2:1,3:7,4:4,7:8}

    for i in range(0,len(test_lines)):
        for j in range(0,4):
            if len(right[i][j]) in [2,3,4,7]:
                four_digits[j] = digit_dict[len(right[i][j])]
                four_digits_fix[j] = True
                if len(right[i][j]) == 2:
                    actual_c_f = right[i][j]
            elif len(right[i][j]) == 5:
                group_five.append(j)
            else: 
                group_six.append(j)
            if sum(four_digits_fix) != 4:
                for k in range(0,10):
                    for char in range(0,7):
                        if a_g[char] in left[i][k]: 
                            count_char[char] += 1
                    if len(left[i][k]) == 2:
                        actual_c_f = left[i][k]

                three_char = 0
                for char in range(0,7):
                    if count_char[char] == 4:
                        actual_e = a_g[char]
                        three_char += 1
                    elif count_char[char] == 6:
                        actual_b = a_g[char]
                        three_char += 1
                    elif count_char[char] == 9:
                        actual_f = a_g[char]
                        three_char += 1
                    if three_char == 3:
                        break

                if len(group_five) > 0:
                    for elem in group_five:
                        if actual_b in right[i][elem]:
                            four_digits[elem] = 5
                            four_digits_fix[elem] = True
                        elif actual_e in right[i][elem]:
                            four_digits[elem] = 2
                            four_digits_fix[elem] = True
                        else:
                            four_digits[elem] = 3
                            four_digits_fix[elem] = True
                if len(group_six) > 0:
                    for elem in group_six:
                        if actual_e not in right[i][elem]:
                            four_digits[elem] = 9
                            four_digits_fix[elem] = True
                        elif actual_c_f[0] in right[i][elem] and actual_c_f[1] in right[i][elem]: # the order of alphabet can be switched!!
                            four_digits[elem] = 0
                            four_digits_fix[elem] = True
                        else:
                            four_digits[elem] = 6
                            four_digits_fix[elem] = True   

        output = four_digits[0]*1000+four_digits[1]*100+four_digits[2]*10+four_digits[3] 
        output_values.append(output)

        count_char = [0] * 7
        four_digits_fix = [bool(False)] * 4
        four_digits = [0] * 4
        group_five = []
        group_six = []

    # print(output_values)
    return(sum(output_values))

In [99]:
sum_actual_output_values('input.txt')

1019355